# Generating Gamma Surfaces & Stacking Faults

Gamma surfaces (also called generalised stacking faults) and stacking faults are forms of crystal defects formed by sliding two slabs of a crystal against each other along a given plane. Often, these defects have metastable states (also called stacking faults), which can exist at grain boundaries or between two partial dislocations. Here, we deal with the full pathway from a complete crystal, sliding through any metastable points until we return back to the perfect crystal again. 

For a stacking fault, the slabs are moved along a single line, whereas for gamma surfaces this is the full plane (hence generalised stacking fault). `matscipy.gamma_surface` provides two classes - `GammaSurface` and `StackingFault` - in order to model these systems, including utility methods for relaxing and plotting the associated potential energy surfaces.

## Stacking Faults for a Specific Dislocation
When modelling systems of dissociated dislocations connected by a stacking fault, it can be useful to see the stacking fault in isolation. To do this, we will use `matscipy.gamma_surface.StackingFault` to model the stacking fault associated with the dissociated `matscipy.dislocations.DiamondGlideScrew` dislocation in Carbon. 

We will first use a potential by  to obtain reasonable guesses for a lattice parameter and elastic constants, and take a look at the dissociated `DiamondGlideScrew` dislocation:

In [ ]:
from matscipy.dislocation import DiamondGlideScrew, get_elastic_constants
from matscipy.gamma_surface import StackingFault, GammaSurface
from matscipy.calculators.manybody.explicit_forms.tersoff_brenner import TersoffBrenner, \
                                                                         Brenner_PRB_42_9458_C_I, \
                                                                         Kioseoglou_PSSb_245_1118_AlN
from matscipy.calculators.manybody import Manybody
from visualisation import show_dislocation, show_stacking_fault
import nglview

calc = Manybody(**TersoffBrenner(Brenner_PRB_42_9458_C_I))

# the function accepts any ASE type of calculator
alat, C11, C12, C44 = get_elastic_constants(calculator=calc, symbol="C", verbose=False)
print(f"alat = {alat}")
C_screw = DiamondGlideScrew(alat, C11, C12, C44, symbol="C")

C_screw_bulk, C_screw_dislo = C_screw.build_cylinder(radius=25, partial_distance=20)

view = show_dislocation(C_screw_dislo, 
                 diamond_structure=True, # bonds make it a bit easier to see the structure
                 scale=0.3, add_bonds=True, 
                 partial_distance=20 * C_screw.glide_distance, 
                 d_name="1/6<112> 30 degree partial screw") 

view.control.zoom(0.7)
view

To model just the stacking fault, we can call `StackingFault` with just the lattice constant, the chemical symbol, and the kind of dissociated dislocation, and use the function `StackingFault.generate_images()` to generate the fault images.

In [ ]:
fault = StackingFault(alat, DiamondGlideScrew, symbol="C")
fault.generate_images(n=8) # Generate 8 images along the pathway
view = show_stacking_fault(fault.images, CNA_color=True, scale=0.8)
view.control.zoom(0.1)
view

NOTE: For the purposes of clarity, the plots all show supercells of the images produced by the code. This is because the surfaces in those configurations are at the cell boundaries, and so are less easy to observe without the layers of atoms on the other side of the boundary.

Now that we have the images generated, we can use the calculator by [D. Holland and M. Marder](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.80.746) to relax the images, and look at the energy barrier associated with forming the stacking fault.

There are many things to consider when measuring stacking fault energy densities, and there are many conventions found in literature. Here, we only look at relaxations of these systems, but there are still many parameters to explore. We should consider whether the system should be vacuum terminated, how far apart the periodic images of the stacking fault should be, what constraints to add to the system, and whether there could be another plane with the same normal that could be more energetically favourable.

This is a lot to consider, so we will explore the effect of tuning these parameters now.

### Parameterising stacking fault image generation

The default behaviour of `StackingFault.generate_images()` is to build as small of a cell as possible, based on the output of either `ase.lattice.cubic` (when `a` is a lattice parameter), or `ase.build.bulk`. This is likely to suffer from finite size effects for most systems, so it may be useful to include more bulk between periodic images of a stacking fault. We will also add a vacuum layer between the faults, as this is a common methodology in the literature.

In [ ]:
# Re-running fault.generate_images() will replace the images
fault.generate_images(
    n=8, # Still generate 8 images
    z_replications=3, # Add copies of the bulk between stacking faults
    vacuum=6.0 # Add vacuum layer between stakcing faults
)
view = show_stacking_fault(fault.images, scale=0.8)
view.control.zoom(0.3)
view

We can see that this gives a stacking fault with more bulk, and that there is now a vacuum layer between the stacking faults.
Next, let's relax the fault and look at the energy barrier.

### Relaxing stacking fault images

To relax the stacking fault images, we can use `StackingFault.relax_images`. Here, we have more choices to make, mainly through which constraints we apply to the images, which are controlled by the arguments `constrain_atoms` and `cell_relax`. As the name suggests, `constrain_atoms` adds atomic constraints, which restrict the atoms to only relax in the Z direction (towards or away from the stacking fault). The `cell_relax` argument allows the length of the cell in Z (`image.cell[2, 2]`) to be relaxed.

Because we added a vacuum layer, it is inefficient to try to relax the cell, therefore we will use `cell_relax=False`. We will then use `StackingFault.get_surface_energies()` to evaluate the energy barrier associated with the stacking fault.

NOTE: With stacking faults, we actually measure the energy __density__ (energy per unit area of the stacking fault), which is often expressed in $\text{eV/}\AA^2$ or in $\text{J/m}^2$. We return with $\text{eV/}\AA^2$ in `StackingFault.get_surface_energies()`.

In [ ]:
fault.relax_images(calc, cell_relax=False)
Es = fault.get_surface_energies(calc)

Now that we have relaxed the images and found the energy barrier, we should see what the barrier looks like. One way of doing this is to use the inbuilt `StackingFault.plot_gamma_surface()` function:

In [ ]:
fault.plot_gamma_surface()

We can see that the potential predicts a $\sim 0.40eV/\AA^2$ barrier, a shallow metastable state, and then a new minimum state with formation energy $\sim 0.05eV/\AA^2$ - this new state is exactly the stacking fault we see in the dislocation.

## Gamma Surfaces without a dislocation

All of the above applies to the `GammaSurface` class as well, with the exception that `GammaSurface.generate_images()` requires `nx` and `ny` as arguments, rather than the `n` argument from `StackingFault.generate_images()` (we need to define a grid of points, rather than a line).

When using `GammaSurface` with a dislocation, `ny` will control the number of points in the direction of the stacking fault we see in that dislocation (`GammaSurface.generate_images(nx=1, ny=n)` is essentially equivalent to `StackingFault.generate_images(n=n)`). However, there exist a wide range of stacking faults and gamma surfaces that may not be linked to a dissociated dislocation in `matscipy.dislocation`.

In order to generate stacking faults and gamma surfaces in any desired direction, we need a well-defined normal vector in miller index notation. In this example, we will take the normal to be (001). As a test system, lets use FCC Al and the potential by [Kioseoglou _et. al._]().

In [ ]:

calc = Manybody(**TersoffBrenner(Kioseoglou_PSSb_245_1118_AlN))

alat, C11, C12, C44 = get_elastic_constants(calculator=calc, symbol="Al", verbose=False)
surface = GammaSurface(alat, surface_direction=[0, 0, 1], crystalstructure="fcc", symbol="Al")

To do a gamma surface or stacking fault calculation, we need a complete orthogonal basis, however we only supplied a single basis vector. Internally, the code has searched for a reasonable choice of basis on the assumption that "good" bases have integer miller indeces, and that the size of these integers should be close to zero.

To see what the full basis is in miller index notation, we can look at the `surf_directions` dict:

In [ ]:
print(surface.surf_directions)

The (001) direction we set as the normal to the surface is chosen to be in the z direction. The code has then found that (010) and (-100) are two directions that result in a "sensible" basis for performing out calculations. 

In the `get_surface_energies()` function, there is an optional argument `relax` which internally calls `relax_images`. This means we can relax the images and get associated energy densities all from one function call.

In [ ]:
surface.generate_images(nx=5,
                        ny=5,
                        z_replications=2)

Es = surface.get_surface_energies(calc, relax=True)

Now that the images are relaxed, we can plot the energy density of the system across the full surface. We will also use the `si=True` argument in the plotting function to convert the energy densities from $eV/\AA^2$ into $J/m^2$.

In [ ]:
surface.plot_gamma_surface(si=True)

We can see that the plot has helpfully labelled the axes with the miller indices, and from this we can see that the potential thinks that the system will want to glide in the (110) direction.4

We can test this further by giving the `GammaSurface` class a more explicit basis using the `y_dir` argument:

In [ ]:
surface = GammaSurface(alat, surface_direction=[0, 0, 1], y_dir=[1, 1, 0], crystalstructure="fcc", symbol="Al")
surface.generate_images(nx=5,
                        ny=5,
                        z_replications=2)

Es = surface.get_surface_energies(calc, relax=True)
surface.plot_gamma_surface(si=True)



## Stacking Faults and Gamma Surfaces in Multi-species Systems
The `GammaSurface` and `StackingFault` classes handle input arguments in a similar way to the classes in `matscipy.disocation`, in that instead of supplying a lattice parameter + crystalstructure + symbol, we can instead supply a bulk structure in the form of an Atoms object. This use-case follows similar caveats to the dislocation systems in complex systems, in that we are limited to systems that can be expressed in a cubic cell. Generating stacking faults from dislocations (as in the first section) also requires that the input bulk shares geometry with the "base" crystalstructure of that dislocation (e.g. the DiamondGlideScrew example will be fine in crystal systems like Diamond and Zincblende, as Zincblende has equivalent geometry).

To repeat the GaAs example from the multispecies dislocation documentation:

In [ ]:
from ase.build import bulk

# Data from https://doi.org/10.1080/08927022.2011.602975
alat = 11.2402/2

GaAs = bulk("GaAs", crystalstructure="zincblende", cubic=True, a=alat)

surface = StackingFault(GaAs, DiamondGlideScrew)

surface.generate_images(32)

view = show_stacking_fault(surface.images, scale=0.8)
view.control.zoom(0.2)
view